In [1]:
%pip install -U imbalanced-learn


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) #MOSTRAR TODAS LAS COLUMNAS

In [3]:
!gdown --id 1-tNBUCWFU0_b9QW6ie4Vvfs7eb4WbIPY
# https://drive.google.com/file/d/1-tNBUCWFU0_b9QW6ie4Vvfs7eb4WbIPY/view?usp=share_link
!gdown --id 11eGwBDC3FoJh8eanfpYXFAS-3SZ6vNnO
#https://drive.google.com/file/d/11eGwBDC3FoJh8eanfpYXFAS-3SZ6vNnO/view?usp=sharing

/opt/homebrew/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1-tNBUCWFU0_b9QW6ie4Vvfs7eb4WbIPY
From (redirected): https://drive.google.com/uc?id=1-tNBUCWFU0_b9QW6ie4Vvfs7eb4WbIPY&confirm=t&uuid=7bf5b457-38f9-4338-b171-52b7415614fe
To: /Users/davidnunez/Desktop/tesis/tesis-model/TestBalanceo1.csv
100%|████████████████████████████████████████| 428M/428M [00:15<00:00, 27.8MB/s]
/opt/homebrew/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=11eGwBDC3FoJh8eanfpYXFAS-3SZ6vNnO
To: /Users/davidnunez/Desktop/tesis/tesis-model/OcidConBanderas.csv
100%|███████████████████████

In [30]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df_completo = pd.read_csv('TestBalanceo1.csv', sep=";;;", index_col=False)

/var/folders/6s/z3758tx16yd17yy35vnsnx_80000gn/T/ipykernel_33562/201650735.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_completo = pd.read_csv('TestBalanceo1.csv', sep=";;;", index_col=False)


In [31]:
df_banderas_completo = pd.read_csv('OcidConBanderas.csv', sep=";", index_col=False)
df_banderas_completo.drop(['Unnamed: 10'], inplace=True, axis=1)
df_completo = df_completo.merge(df_banderas_completo, left_on='ocid', right_on='id', how='left')

In [55]:
#Drop
df_dummizado = df_completo.drop(['ocid', 'id_x', 'id_y', 'tender.id', 'tender.bidOpening.address.streetAddress', 'tender.enquiriesAddress.streetAddress', 'ocid.1', 'tender.procurementIntention.uri',
    'tender.procurementIntention.description', 'tender.procurementIntention.category', 'tender.procurementIntention.title', 'initiationType','tender.procurementIntention.id', # Ver vien por que dropeamos
    'tag', 'language'], axis=1)

#Dummies
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.status']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.awardCriteria']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.awardCriteriaDetails']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.statusDetails']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.hasEnquiries']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.value.currency']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.mainProcurementCategory']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.procurementMethod']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['planning.budget.amount.currency']) # Cardinalidad baja

df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.procurementIntention.status']) # Cardinalidad baja
df_dummizado = pd.get_dummies(df_dummizado, columns=['tender.procurementIntention.statusDetails']) # Cardinalidad baja


#Dates
df_dummizado['tender.bidOpening.date.month'] = pd.to_datetime(df_dummizado['tender.bidOpening.date'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.bidOpening.date.year'] = pd.to_datetime(df_dummizado['tender.bidOpening.date'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.bidOpening.date.yearmonth'] = pd.to_datetime(df_dummizado['tender.bidOpening.date'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%m%Y')

df_dummizado['tender.datePublished.month'] = pd.to_datetime(df_dummizado['tender.datePublished'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.datePublished.year'] = pd.to_datetime(df_dummizado['tender.datePublished'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.datePublished.yearmonth'] = pd.to_datetime(df_dummizado['tender.datePublished'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%m%Y')

df_dummizado['tender.tenderPeriod.startDate.month'] = pd.to_datetime(df_dummizado['tender.tenderPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.tenderPeriod.startDate.year'] = pd.to_datetime(df_dummizado['tender.tenderPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.tenderPeriod.startDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.tenderPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%m%Y')

df_dummizado['tender.tenderPeriod.endDate.month'] = pd.to_datetime(df_dummizado['tender.tenderPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.tenderPeriod.endDate.year'] = pd.to_datetime(df_dummizado['tender.tenderPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.tenderPeriod.endDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.tenderPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%m%Y')

df_dummizado['tender.awardPeriod.startDate.month'] = pd.to_datetime(df_dummizado['tender.awardPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.awardPeriod.startDate.year'] = pd.to_datetime(df_dummizado['tender.awardPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.awardPeriod.startDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.awardPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%m%Y')

df_dummizado['tender.enquiryPeriod.endDate.month'] = pd.to_datetime(df_dummizado['tender.enquiryPeriod.endDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.enquiryPeriod.endDate.year'] = pd.to_datetime(df_dummizado['tender.enquiryPeriod.endDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.enquiryPeriod.endDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.enquiryPeriod.endDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%m%Y')

df_dummizado['tender.enquiryPeriod.startDate.month'] = pd.to_datetime(df_dummizado['tender.enquiryPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.enquiryPeriod.startDate.year'] = pd.to_datetime(df_dummizado['tender.enquiryPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.enquiryPeriod.startDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.enquiryPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%m%Y')

df_dummizado['date.month'] = pd.to_datetime(df_dummizado['date'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['date.year'] = pd.to_datetime(df_dummizado['date'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['date.yearmonth'] = pd.to_datetime(df_dummizado['date'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%m%Y')

df_dummizado['planning.estimatedDate.month'] = pd.to_datetime(df_dummizado['planning.estimatedDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['planning.estimatedDate.year'] = pd.to_datetime(df_dummizado['planning.estimatedDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['planning.estimatedDate.yearmonth'] = pd.to_datetime(df_dummizado['planning.estimatedDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%m%Y')

df_dummizado['tender.contractPeriod.maxExtentDate.month'] = pd.to_datetime(df_dummizado['tender.contractPeriod.maxExtentDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.contractPeriod.maxExtentDate.year'] = pd.to_datetime(df_dummizado['tender.contractPeriod.maxExtentDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.contractPeriod.maxExtentDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.contractPeriod.maxExtentDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%m%Y')

df_dummizado['tender.procurementIntention.startDate.month'] = pd.to_datetime(df_dummizado['tender.procurementIntention.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.procurementIntention.startDate.year'] = pd.to_datetime(df_dummizado['tender.procurementIntention.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.procurementIntention.startDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.procurementIntention.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%m%Y')

df_dummizado['tender.procurementIntention.publishedDate.month'] = pd.to_datetime(df_dummizado['tender.procurementIntention.publishedDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.procurementIntention.publishedDate.year'] = pd.to_datetime(df_dummizado['tender.procurementIntention.publishedDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.procurementIntention.publishedDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.procurementIntention.publishedDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%m%Y')

df_dummizado['tender.contractPeriod.startDate.month'] = pd.to_datetime(df_dummizado['tender.contractPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.contractPeriod.startDate.year'] = pd.to_datetime(df_dummizado['tender.contractPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.contractPeriod.startDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.contractPeriod.startDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%m%Y')

df_dummizado['tender.contractPeriod.endDate.month'] = pd.to_datetime(df_dummizado['tender.contractPeriod.endDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.month
df_dummizado['tender.contractPeriod.endDate.year'] = pd.to_datetime(df_dummizado['tender.contractPeriod.endDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.year
df_dummizado['tender.contractPeriod.endDate.yearmonth'] = pd.to_datetime(df_dummizado['tender.contractPeriod.endDate'], format='%Y-%m-%dT%H:%M:%S-04:00').dt.strftime('%m%Y')

# Dropeando data interesante (Despues ver como procesar, son datos practicamente unicos)
df_dummizado = df_dummizado.drop(['tender.title', 'tender.submissionMethodDetails', 'tender.eligibilityCriteria', 'tender.mainProcurementCategoryDetails', 'planning.budget.description', ], axis=1)

# Dropeando por cardinalidad alta? Datos no 100% unicos por cada row
df_dummizado = df_dummizado.drop(['tender.procurementMethodDetails', 'tender.procuringEntity.id', 'tender.procuringEntity.name', 'buyer.id', 'buyer.name', 
	'tender.procurementIntention.procuringEntity.id', 'tender.procurementIntention.procuringEntity.name', 'secondStage.id',
], axis=1)

# Otras conversiones
df_dummizado['tender.techniques.hasElectronicAuction'] = np.where(df_dummizado['tender.techniques.hasElectronicAuction'] == True, 1, 0)


In [60]:

#Dropeando dates
df_dummizado = df_dummizado.drop(['tender.bidOpening.date',
	'tender.datePublished', 'tender.tenderPeriod.startDate', 'tender.tenderPeriod.endDate', 'tender.awardPeriod.startDate','tender.enquiryPeriod.endDate','tender.enquiryPeriod.startDate',
    'date', 'planning.estimatedDate', 'tender.contractPeriod.maxExtentDate','tender.procurementIntention.startDate', 'tender.procurementIntention.publishedDate',
	'tender.contractPeriod.startDate', 'tender.contractPeriod.endDate'
], axis= 1)

In [ ]:
# GROUP BY EXAMPLE
gkk = df_completo.groupby(['secondStage.id'], dropna=False)
gkk.count()

In [23]:
df_complaints = df_dummizado[(df_dummizado['has_complaint'] == True)]
df_no_complaints = df_dummizado[(df_dummizado['has_complaint'] == False)]

In [24]:

df_complaints.shape[0]

10332

In [63]:
(df_complaints.shape[0] * 2) / df_no_complaints.shape[0]

0.2377085010928333

In [64]:
from sklearn.utils import resample
# from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

In [65]:

X = df_dummizado.drop('has_complaint', axis=1)  # Features (all columns except 'target')
y = df_dummizado['has_complaint']  # Target variable
print('Original dataset shape %s' % Counter(y))
# Original dataset shape Counter({1: 900, 0: 100})
smote = SMOTE(random_state=42, sampling_strategy= 0.3)
X_resampled, y_resampled = smote.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_resampled))


Original dataset shape Counter({False: 86930, True: 10332})


ValueError: could not convert string to float: 'covid-19'